Deep Learning
=============

Assignment 2
------------

Previously in `1_notmnist.ipynb`, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [12]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

First reload the data we generated in `1_notmnist.ipynb`.

In [13]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (10000, 28, 28) (10000,)
Validation set (1000, 28, 28) (1000,)
Test set (1000, 28, 28) (1000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [14]:
#Convertir todo a Arrays 2D
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (10000, 784) (10000, 10)
Validation set (1000, 784) (1000, 10)
Test set (1000, 784) (1000, 10)


We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:
* First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:

      with graph.as_default():
          ...

* Then you can run the operations on this graph as many times as you want by calling `session.run()`, providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:

      with tf.Session(graph=graph) as session:
          ...

Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

In [52]:
###AQUÍ SE PREPARA LA NEURAL NETWORK###

'''
First you describe the computation that you want to see performed: what the inputs, the variables, 
and the operations look like. These get created as nodes over a computation graph.
'''

'''
tf.Graph: A TensorFlow computation, represented as a dataflow graph. <-- Aqui comienza todo
graph.as_default: Returns a context manager that makes this `Graph` the default graph.
'''
graph = tf.Graph()
with graph.as_default():

    ### Input data ###
    # Load the training, validation and test data into constants that are attached to the graph.
    tf_train_dataset = tf.constant(train_dataset)
    tf_train_labels = tf.constant(train_labels)
    print('Train as a TF Constant:',tf_train_dataset)
    print('Train labels as a TF Constant:',tf_train_labels,'\n')

    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    ### Variables ###
    # These are the parameters that we are going to be training. 
    # The weight matrix will be initialized using random values following a (truncated) normal distribution.
    # The biases get initialized to zero.
    
    normalDist = tf.truncated_normal(shape=[image_size * image_size, num_labels])#Outputs random values from a truncated normal distribution.
    print('Truncated Normal Distribution: ',normalDist)
    weights = tf.Variable(normalDist)
    print('Weights: ',weights,'\n')
    
    zeros = tf.zeros([num_labels])
    print('Zeros: ',zeros)
    biases = tf.Variable(zeros)
    print('Biases: ',biases,'\n')

    
    ### Training computation ###
    # We multiply the inputs with the weight matrix, and add biases. 
    # We compute the softmax and cross-entropy (it's one operation in TensorFlow, because it's very common, and it can be optimized).
    # We take the average of this cross-entropy across all training examples --> that's our loss.
    '''
    tf.matmul: Multiplies matrix `a` by matrix `b`, producing `a` * `b`.
    '''
    
    logits = tf.matmul(tf_train_dataset, weights) + biases #Multiplicar input con matriz de pesos y sumamos bias
    print('Logits: ',logits,'\n')
    
    '''
    tf.nn.softmax_cross_entropy_with_logits: Computes softmax cross entropy between `logits` and `labels`.
    tf.reduce_mean: Computes the mean of elements across dimensions of a tensor.
    '''
    
 
    softMaxCrossEntropy = tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)
    print('softMaxCrossEntropy: ',softMaxCrossEntropy)
    loss = tf.reduce_mean(softMaxCrossEntropy)
    print('Loss: ',loss)

    ### Optimizer ###
    # We are going to find the minimum of this loss using gradient descent.
    '''
    tf.train.GradientDescentOptimizder: Optimizer that implements the gradient descent algorithm.
    GradientDescentOptimizder.minimize: Add operations to minimize `loss` by updating `var_list`.
    '''
    GDOptimizer = tf.train.GradientDescentOptimizer(0.5)
    print('GDOptimizer: ',GDOptimizer)
    optimizer = GDOptimizer.minimize(loss)
    print('Optimizer: ',optimizer.name,'\n')
    
    ### Predictions for the training, validation, and test data ###
    # These are not part of training, but merely here so that we can report accuracy figures as we train.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
    
    print('Train prediction: ',train_prediction)
    print('Valid prediction: ',valid_prediction)
    print('Test prediction: ',test_prediction)

Train as a TF Constant: Tensor("Const:0", shape=(10000, 784), dtype=float32)
Train labels as a TF Constant: Tensor("Const_1:0", shape=(10000, 10), dtype=float32) 

Truncated Normal Distribution:  Tensor("truncated_normal:0", shape=(784, 10), dtype=float32)
Weights:  Tensor("Variable/read:0", shape=(784, 10), dtype=float32) 

Zeros:  Tensor("zeros:0", shape=(10,), dtype=float32)
Biases:  Tensor("Variable_1/read:0", shape=(10,), dtype=float32) 

Logits:  Tensor("add:0", shape=(10000, 10), dtype=float32) 

softMaxCrossEntropy:  Tensor("Reshape_2:0", shape=(10000,), dtype=float32)
Loss:  Tensor("Mean:0", shape=(), dtype=float32)
GDOptimizer:  <tensorflow.python.training.gradient_descent.GradientDescentOptimizer object at 0x000001936E289A58>
Optimizer:  GradientDescent 

Train prediction:  Tensor("Softmax:0", shape=(10000, 10), dtype=float32)
Valid prediction:  Tensor("Softmax_1:0", shape=(1000, 10), dtype=float32)
Test prediction:  Tensor("Softmax_2:0", shape=(1000, 10), dtype=float32)


Let's run this computation and iterate:

In [53]:
###AQUÍ SE ENTRENA Y EVALÚA LA NEURAL NETWORK###
num_steps = 801

#Calcula la accuracy
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

'''
Then you can run the operations on this graph as many times as you want by calling session.run(), 
providing it outputs to fetch from the graph that get returned. 
This runtime operation is all contained in the block below:
'''

'''
tf.Session: A class for running TensorFlow operations.
'''
with tf.Session(graph=graph) as session:
    # This is a one-time operation which ensures the parameters get initialized as we described in the graph
    tf.global_variables_initializer().run()
    '''Si no se ejecuta --> FailedPreconditionError (see above for traceback): Attempting to use uninitialized value'''
    print('Initialized')
    
    # Run the computations.
    for step in range(num_steps):
        '''
        session.run: Runs operations and evaluates tensors in `fetches`.
        returns:  Either a single value if `fetches` is a single graph element, or
                  a list of values if `fetches` is a list, or a dictionary with the
                  same keys as `fetches` if that is a dictionary (described above).
        '''
        # We tell .run() that we want to run the optimizer, and get the loss value and the training predictions
        
        #Nuestro 'fetches' es una lista con [ElOptimizer, El Loss, Las predicciones] y lo mismo nos retornará
        dummyOpt, returnedLoss, returnedPred = session.run(fetches=[optimizer, loss, train_prediction])
        
        if (step % 100 == 0):
            print('Loss at step ',step,':',returnedLoss)
            
            acc = accuracy(returnedPred,train_labels)
            print('Training accuracy: ',acc)
            
            '''
            prediction.eval: Evaluates this tensor in a `Session`. Calling this method will execute all preceding operations 
                             that produce the inputs needed for the operation that produces this tensor.
            '''
            # Calling .eval() on valid_prediction is basically like calling run(), but just to get that one returned value.
            # Note that it recomputes all its graph dependencies.
            #VALID#
            validPred = valid_prediction.eval()
            validAcc = accuracy(validPred,valid_labels)
            print('Validation accuracy: ',validAcc)
            #TEST#
            testPred = test_prediction.eval()
            testAcc = accuracy(testPred,test_labels)
            print('Validation accuracy: ',testAcc)
            

Initialized
Loss at step  0 : 14.9884
Training accuracy:  16.46
Validation accuracy:  19.4
Validation accuracy:  21.4
Loss at step  100 : 2.37722
Training accuracy:  72.37
Validation accuracy:  73.0
Validation accuracy:  78.5
Loss at step  200 : 1.93734
Training accuracy:  74.99
Validation accuracy:  75.5
Validation accuracy:  81.1
Loss at step  300 : 1.69569
Training accuracy:  75.98
Validation accuracy:  76.2
Validation accuracy:  81.4
Loss at step  400 : 1.52879
Training accuracy:  76.59
Validation accuracy:  76.0
Validation accuracy:  81.7
Loss at step  500 : 1.4024
Training accuracy:  76.96
Validation accuracy:  76.5
Validation accuracy:  81.9
Loss at step  600 : 1.30152
Training accuracy:  77.49
Validation accuracy:  76.8
Validation accuracy:  82.4
Loss at step  700 : 1.2183
Training accuracy:  77.99
Validation accuracy:  76.5
Validation accuracy:  82.6
Loss at step  800 : 1.14816
Training accuracy:  78.37
Validation accuracy:  76.7
Validation accuracy:  82.4


Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a `Placeholder` node which will be fed actual data at every call of `session.run()`.

In [68]:
###AQUÍ SE PREPARA LA NEURAL NETWORK PERO PARA STOCHASTIC GD###

batch_size = 128

graph = tf.Graph()
with graph.as_default():

    ###DATA###
    # Input data. For the training data, we use a placeholder that will be fed at run time with a training minibatch.
    '''
    En este caso, en vez de usar un 'Constant', es decir, el set de datos completo, usamos un 'Placeholder', que pilla el 
    set de datos completo pero está preparado para luego entregárselo al Graph en 'trocitos' o 'minibatches' del batch_size indicado
    '''
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    ### Variables ### - Igual que antes
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))

    ### Training computation ### - Igual que antes
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    ### Optimizer ### - Igual que antes
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    ### Predictions for the training, validation, and test data ### - Igual que antes
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
    
print('Estructura de la Red Neuronal creada.\nLa estructura es igual salvo que los datos están almacenados en PlaceHolders'
      +'en vez de en Constants, para poder ser fragmentados en MiniBatches')

Estructura de la Red Neuronal creada.
La estructura es igual salvo que los datos están almacenados en PlaceHoldersen vez de en Constants, para poder ser fragmentados en MiniBatches


Let's run it:

In [74]:
###AQUÍ SE ENTRENA Y EVALÚA LA NEURAL NETWORK PERO CON STOCHASTIC GD###

#Al usar Stochastic podemos aumentar mucho el nº de epochs sin sufrir en rendimiento
num_steps = 3001

with tf.Session(graph=graph) as session:
    #Inicializar la estructura que hemos definido anteriormente - Igual que en el anterior
    tf.global_variables_initializer().run()
    print("Initialized")
    
    for step in range(num_steps):
        #Pick an offset within the training data, which has been randomized. Note:We could use better randomization across epochs.
        #Lo curioso de este método es que son aleatorios pero van incrementando hasta llegar al tamaño del set y vuelve a empezar
        batchStartIndex = (step * batch_size) % (train_labels.shape[0] - batch_size) #Por cada paso pillamos batches aleatorios
        batchEndIndex = batchStartIndex + batch_size
        
        # Generate a minibatch.
        batch_data = train_dataset[batchStartIndex:batchEndIndex, :] #[Comienzo : final] --> [batchStartIndex : batchStartIndex + tamañoBatch]
        batch_labels = train_labels[batchStartIndex:batchEndIndex, :]
        
        # Prepare a dictionary telling the session where to feed the minibatch.
        '''
        KEY: of the dictionary is the placeholder node of the graph to be fed,
        VALUE: is the numpy array to feed to it.
        '''
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        
        #Igual que en el GD normal pero
        #Hay que usar el parámetro 'feed_dict' e indicarle el dict que hemos creado para decirle a la sesión cómo encontrar los datos
        dummyOpt, returnedLoss, returnedPred = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            
            print('Fragmento/MiniBatch loss at step ',step,': ',returnedLoss)
            
            acc = accuracy(returnedPred,batch_labels)
            print('Fragmento/MiniBatch accuracy: ',acc)
            
            validPred = valid_prediction.eval()
            validAcc = accuracy(validPred,valid_labels)
            print('Validation accuracy: ',validAcc)
            
            testPred = test_prediction.eval()
            testAcc = accuracy(testPred,test_labels)
            print('Test accuracy: ',testAcc)
            

Initialized
Fragmento/MiniBatch loss at step  0 :  16.3362
Fragmento/MiniBatch accuracy:  12.5
Validation accuracy:  13.6
Test accuracy:  14.9
Fragmento/MiniBatch loss at step  500 :  1.35914
Fragmento/MiniBatch accuracy:  75.0
Validation accuracy:  76.9
Test accuracy:  81.4
Fragmento/MiniBatch loss at step  1000 :  1.0519
Fragmento/MiniBatch accuracy:  75.0
Validation accuracy:  76.5
Test accuracy:  82.1
Fragmento/MiniBatch loss at step  1500 :  0.903583
Fragmento/MiniBatch accuracy:  79.6875
Validation accuracy:  76.9
Test accuracy:  82.7
Fragmento/MiniBatch loss at step  2000 :  0.577678
Fragmento/MiniBatch accuracy:  82.03125
Validation accuracy:  76.8
Test accuracy:  84.5
Fragmento/MiniBatch loss at step  2500 :  0.89757
Fragmento/MiniBatch accuracy:  85.9375
Validation accuracy:  77.7
Test accuracy:  84.1
Fragmento/MiniBatch loss at step  3000 :  0.444482
Fragmento/MiniBatch accuracy:  86.71875
Validation accuracy:  77.5
Test accuracy:  84.7


---
Problem
-------

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units [nn.relu()](https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#relu) and 1024 hidden nodes. This model should improve your validation / test accuracy.

---

In [86]:
###AQUÍ SE PREPARA LA NEURAL NETWORK CON SGD PERO CON RELU###

batch_size = 128

graph = tf.Graph()
with graph.as_default():

    ###DATA###
    # Input data. For the training data, we use a placeholder that will be fed at run time with a training minibatch.
    '''
    En este caso, en vez de usar un 'Constant', es decir, el set de datos completo, usamos un 'Placeholder', que pilla el 
    set de datos completo pero está preparado para luego entregárselo al Graph en 'trocitos' o 'minibatches' del batch_size indicado
    '''
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    ### Variables & Layers ###
    '''Sinapsis Input a Hidden'''
    weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, 1024]))
    biases1 = tf.Variable(tf.zeros([1024]))
    '''Sinapsis Hidden a Output'''
    weights2 = tf.Variable(tf.truncated_normal([1024, num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))

    '''Hidden Layer'''
    hidden1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
    
    ### Training computation ###
    logits = tf.matmul(hidden1, weights2) + biases2 #Logits: La última capa y los útlimos weights y bias
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) #Igual

    
    ### Optimizer ### - Igual que antes
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    ### Predictions for the training, validation, and test data ###
    train_prediction = tf.nn.softmax(logits) #Igual
    
    #VALID# 
    inputToHiddenMultVal = tf.matmul(tf_valid_dataset, weights1) + biases1 #Multiplicamos input x weights y sumamos bias
    hiddenToOutputReLUVal = tf.nn.relu(inputToHiddenMultVal + biases1) #Añadimos ReLU y sumamos bias
    completeMultVal = tf.matmul(hiddenToOutputReLUVal,weights2) + biases2 #Multiplicamos Hidden x weights2 y sumamos bias2
    valid_prediction = tf.nn.softmax(completeMultVal)
    
    #TEST# 
    inputToHiddenMultTest = tf.matmul(tf_test_dataset, weights1) + biases1 #Multiplicamos input x weights y sumamos bias
    hiddenToOutputReLUTest = tf.nn.relu(inputToHiddenMultTest + biases1) #Añadimos ReLU y sumamos bias
    completeMultTest = tf.matmul(hiddenToOutputReLUTest,weights2) + biases2 #Multiplicamos Hidden x weights2 y sumamos bias2
    test_prediction = tf.nn.softmax(completeMultTest)
    
    
print('Estructura de la Red Neuronal creada.')

Estructura de la Red Neuronal creada.


In [87]:
###AQUÍ SE ENTRENA Y EVALÚA LA NEURAL NETWORK CON SGD Y HIDDEN LAYER ###

#Al usar Stochastic podemos aumentar mucho el nº de epochs sin sufrir en rendimiento
num_steps = 3001

with tf.Session(graph=graph) as session:
    #Inicializar la estructura que hemos definido anteriormente - Igual que en el anterior
    tf.global_variables_initializer().run()
    print("Initialized")
    
    for step in range(num_steps):
        #Pick an offset within the training data, which has been randomized. Note:We could use better randomization across epochs.
        #Lo curioso de este método es que son aleatorios pero van incrementando hasta llegar al tamaño del set y vuelve a empezar
        batchStartIndex = (step * batch_size) % (train_labels.shape[0] - batch_size) #Por cada paso pillamos batches aleatorios
        batchEndIndex = batchStartIndex + batch_size
        
        # Generate a minibatch.
        batch_data = train_dataset[batchStartIndex:batchEndIndex, :] #[Comienzo : final] --> [batchStartIndex : batchStartIndex + tamañoBatch]
        batch_labels = train_labels[batchStartIndex:batchEndIndex, :]
        
        # Prepare a dictionary telling the session where to feed the minibatch.
        '''
        KEY: of the dictionary is the placeholder node of the graph to be fed,
        VALUE: is the numpy array to feed to it.
        '''
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        
        #Igual que en el GD normal pero
        #Hay que usar el parámetro 'feed_dict' e indicarle el dict que hemos creado para decirle a la sesión cómo encontrar los datos
        dummyOpt, returnedLoss, returnedPred = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            
            print('Fragmento/MiniBatch loss at step ',step,': ',returnedLoss)
            
            acc = accuracy(returnedPred,batch_labels)
            print('Fragmento/MiniBatch accuracy: ',acc)
            
            validPred = valid_prediction.eval()
            validAcc = accuracy(validPred,valid_labels)
            print('Validation accuracy: ',validAcc)
            
            testPred = test_prediction.eval()
            testAcc = accuracy(testPred,test_labels)
            print('Test accuracy: ',testAcc)
            

Initialized
Fragmento/MiniBatch loss at step  0 :  372.005
Fragmento/MiniBatch accuracy:  9.375
Validation accuracy:  24.1
Test accuracy:  24.9
Fragmento/MiniBatch loss at step  500 :  10.3998
Fragmento/MiniBatch accuracy:  94.53125
Validation accuracy:  80.4
Test accuracy:  85.2
Fragmento/MiniBatch loss at step  1000 :  1.38454
Fragmento/MiniBatch accuracy:  97.65625
Validation accuracy:  82.6
Test accuracy:  88.2
Fragmento/MiniBatch loss at step  1500 :  0.245282
Fragmento/MiniBatch accuracy:  98.4375
Validation accuracy:  82.6
Test accuracy:  88.3
Fragmento/MiniBatch loss at step  2000 :  0.0892306
Fragmento/MiniBatch accuracy:  99.21875
Validation accuracy:  82.6
Test accuracy:  88.1
Fragmento/MiniBatch loss at step  2500 :  0.0
Fragmento/MiniBatch accuracy:  100.0
Validation accuracy:  83.1
Test accuracy:  88.9
Fragmento/MiniBatch loss at step  3000 :  0.858593
Fragmento/MiniBatch accuracy:  99.21875
Validation accuracy:  83.1
Test accuracy:  88.4
